In [ ]:
# default_exp callback.hyperlogger

# HyperparamsLoggerCallback

> API details.

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib as mpl
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#export
import copy
from itertools import zip_longest
from typing import List, Any, Dict, Callable

from IPython.display import clear_output, display, HTML

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

import pytorch_lightning as pl
from pytorch_lightning.core import LightningModule
from pytorch_lightning.metrics import functional as FM
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.utilities import rank_zero_info

In [ ]:
#export
from typing import Optional

from pytorch_lightning.callbacks.base import Callback
from pytorch_lightning.utilities import rank_zero_warn
from pytorch_lightning.utilities.exceptions import MisconfigurationException


class HyperparamsLogger(Callback):
    r"""
    Automatically logs learning rate for learning rate schedulers during training.

    Args:
        logging_interval: set to `epoch` or `step` to log `lr` of all optimizers
            at the same interval, set to `None` to log at individual interval
            according to the `interval` key of each scheduler. Defaults to ``None``.

    Example::

        >>> from pytorch_lightning import Trainer
        >>> from pytorch_lightning.callbacks import LearningRateLogger
        >>> lr_logger = LearningRateLogger(logging_interval='step')
        >>> trainer = Trainer(callbacks=[lr_logger])

    Logging names are automatically determined based on optimizer class name.
    In case of multiple optimizers of same type, they will be named `Adam`,
    `Adam-1` etc. If a optimizer has multiple parameter groups they will
    be named `Adam/pg1`, `Adam/pg2` etc. To control naming, pass in a
    `name` keyword in the construction of the learning rate schdulers

    Example::

        def configure_optimizer(self):
            optimizer = torch.optim.Adam(...)
            lr_scheduler = {'scheduler': torch.optim.lr_schedulers.LambdaLR(optimizer, ...)
                            'name': 'my_logging_name'}
            return [optimizer], [lr_scheduler]
    """
    def __init__(self, logging_interval: Optional[str] = None):
        if logging_interval not in (None, 'step', 'epoch'):
            raise MisconfigurationException(
                'logging_interval should be `step` or `epoch` or `None`.'
            )

        self.logging_interval = logging_interval
        self.lrs = None
        self.mms = None
        self.lr_sch_names = []

    def on_train_start(self, trainer, pl_module):
        """ Called before training, determines unique names for all lr
            schedulers in the case of multiple of the same type or in
            the case of multiple parameter groups
        """
        if not trainer.logger:
            raise MisconfigurationException(
                'Cannot use LearningRateLogger callback with Trainer that has no logger.'
            )

        if not trainer.lr_schedulers:
            rank_zero_warn(
                'You are using LearningRateLogger callback with models that'
                ' have no learning rate schedulers. Please see documentation'
                ' for `configure_optimizers` method.', RuntimeWarning
            )

        # Find names for schedulers
        lr_names = self._find_names(trainer.lr_schedulers)
        momentum_names = self._find_names(trainer.lr_schedulers, 'momentum')
        # Initialize for storing values
        self.lrs = {name: [] for name in lr_names}
        self.mms = {name: [] for name in momentum_names}

    def on_batch_start(self, trainer, pl_module):
        if self.logging_interval != 'epoch':
            interval = 'step' if self.logging_interval is None else 'any'
            latest_stat = self._extract_hps(trainer, interval)

            if trainer.logger is not None and latest_stat:
                trainer.logger.log_metrics(latest_stat, step=trainer.global_step)

    def on_epoch_start(self, trainer, pl_module):
        if self.logging_interval != 'step':
            interval = 'epoch' if self.logging_interval is None else 'any'
            latest_stat = self._extract_hps(trainer, interval)

            if trainer.logger is not None and latest_stat:
                trainer.logger.log_metrics(latest_stat, step=trainer.current_epoch)

    def _extract_hps(self, trainer, interval):
        latest_stat = {}
        for scheduler in trainer.lr_schedulers:
            for name in self.lr_sch_names:
                if scheduler['interval'] == interval or interval == 'any':
                    param_groups = scheduler['scheduler'].optimizer.param_groups
                    if len(param_groups) != 1:
                        for i, pg in enumerate(param_groups):
                            key =  f'{name}/pg{i + 1}'
                            if 'lr' in name:
                                lr = pg['lr']
                                self.lrs[key].append(lr)
                                latest_stat[key] = lr
                            elif 'momentum' in name:
                                val = self._get_momentum(pg)
                                self.mms[key].append(val)
                                latest_stat[key] = val

                    else:
                        if 'lr' in name:
                            self.lrs[name].append(param_groups[0]['lr'])
                            latest_stat[name] = param_groups[0]['lr']
                        elif 'momentum' in name:
                            val = self._get_momentum(param_groups[0])
                            self.mms[name].append(val)
                            latest_stat[name] = val
        return latest_stat

    def _get_momentum(self, pg):
        val = pg.get('momentum')
        if val is None:
            val = pg.get('betas')
            if val is None:
                raise Exception('This optimizer doenst have momentum.')
            val = val[0]
        return val

    def _find_names(self, lr_schedulers, key='lr'):
        # Create uniqe names in the case we have multiple of the same learning
        # rate schduler + multiple parameter groups

        names = []
        for scheduler in lr_schedulers:
            sch = scheduler['scheduler']
            if 'name' in scheduler:
                name = scheduler['name']
            else:
                opt_name = key + '-' + sch.optimizer.__class__.__name__
                i, name = 1, opt_name
                # Multiple schduler of the same type
                while True:
                    if name not in names:
                        break
                    i, name = i + 1, f'{opt_name}-{i}'

            # Multiple param groups for the same schduler
            param_groups = sch.optimizer.param_groups

            if len(param_groups) != 1:
                for i, pg in enumerate(param_groups):
                    temp = f'{name}/pg{i + 1}'
                    names.append(temp)
            else:
                names.append(name)
            self.lr_sch_names.append(name)

        return names
